# Parallelism

In [4]:
import numpy as np
import tensorflow as tf

2023-03-22 13:53:21.520586: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


https://stackoverflow.com/questions/62759940/parallelization-strategies-for-deep-learning

In [3]:
x_train = np.load('Data/x_train_sample.npy')/255
y_train = np.load('Data/y_train_sample.npy')
x_test = np.load('Data/x_test_sample.npy')/255
y_test = np.load('Data/y_test_sample.npy')

## Data Parallelism